## Imports and Initialization of script

In [1]:
!pip install pyspark

import sys
import os

os.environ['HADOOP_HOME'] = "C:\Hadoop"
sys.path.append("C:\Hadoop\bin")

os.environ['JAVA_HOME'] = "C:\Java"
sys.path.append("C:\Java\bin")

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
builder.\
appName("pyspark-notebook2").\
master("local").\
config("spark.executor.memory", "1g").\
config("spark.mongodb.input.uri","mongodb://127.0.0.1/Project6").\
config("spark.mongodb.output.uri","mongodb://127.0.0.1/Project6").\
config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
config("spark.jars", "\Spark\spark-3.3.1-bin-hadoop3\jars\mysql-connector-j-8.0.32.jar").\
getOrCreate()

## Extract

In [28]:
collection_list = ['Research_2007_2013', 'Research_2007_2013_org', 'Research_2007_2013_themes',\
                  'Research_2014_2020', 'Research_2014_2020_org', 'Research_2014_2020_themes',\
                  'Research_2021_2027', 'Research_021_2027_org', 'Research_021_2027_themes',\
                  'OCDE_ind']
    
def extract_collection(collection):
    spark.conf.set('spark.sql.caseSensitive', True)
    
    df_collection = spark.read\
    .format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://localhost:27017/Project6") \
    .option("database", "Project6") \
    .option("collection", collection) \
    .load()
    return df_collection

result = list(map(extract_collection, collection_list))

In [29]:
#testing the extract function
print(result[0])

DataFrame[_id: struct<oid:string>, acronym: string, contentUpdateDate: string, ecMaxContribution: int, endDate: timestamp, frameworkProgramme: string, fundingScheme: string, id: string, legalBasis: string, nature: string, objective: string, rcn: string, startDate: timestamp, status: string, subCall: string, title: string, topics: string, totalCost: double]


In [30]:
result[0].printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- acronym: string (nullable = true)
 |-- contentUpdateDate: string (nullable = true)
 |-- ecMaxContribution: integer (nullable = true)
 |-- endDate: timestamp (nullable = true)
 |-- frameworkProgramme: string (nullable = true)
 |-- fundingScheme: string (nullable = true)
 |-- id: string (nullable = true)
 |-- legalBasis: string (nullable = true)
 |-- nature: string (nullable = true)
 |-- objective: string (nullable = true)
 |-- rcn: string (nullable = true)
 |-- startDate: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- subCall: string (nullable = true)
 |-- title: string (nullable = true)
 |-- topics: string (nullable = true)
 |-- totalCost: double (nullable = true)



## Transform

In [4]:
from pyspark.sql.functions import sum,avg,max,count, round, col, asc

df_time_sum = (df_OCDE_ind.groupby('TIME')\
.agg(sum('Value')\
.alias('sum'))\
.sort(col('TIME')\
.desc()))

df_time_sum.show()

+----+--------------------+
|TIME|                 sum|
+----+--------------------+
|2019| 6.78890384967321E10|
|2018|2.129130072401470...|
|2017|3.944693233941146E11|
|2016|3.759454383108052E11|
|2015| 3.64124148940675E11|
|2014|3.511158262619162...|
|2013|3.402527159873142E11|
|2012|3.293067033760887E11|
|2011|3.189067274586897...|
|2010|2.963322559956424...|
+----+--------------------+



## Load

In [5]:
df_time_sum.write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .option("url", "jdbc:mysql://localhost:3306/Project6") \
  .option("dbtable", "OCDE_ind_year") \
  .option("user", "root") \
  .option("password", "M1a2r3t4a5!") \
  .save()